In [1]:
from bs4 import BeautifulSoup
import urllib.request as req
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd

In [2]:
year = ['2017','2018']
month = ['0{}'.format(i) for i in range(1,10)] + [i for i in range(10,13)]

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome('chromedriver', chrome_options=options)
# driver = webdriver.Chrome('chromedriver')

/home/jongsky/anaconda3/envs/python_3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [4]:
url = 'http://www.weatheri.co.kr/bygone/bygone01.php'

In [5]:
driver.get(url)

In [6]:
# select.select_by_visible_text('2017')

select = Select(driver.find_element_by_name('s_year'))
select.select_by_value('2017')

select = Select(driver.find_element_by_name('s_month'))
select.select_by_value('1')

select = Select(driver.find_element_by_name('s_day'))
select.select_by_value('1')

select = Select(driver.find_element_by_name('e_year'))
select.select_by_value('2018')

select = Select(driver.find_element_by_name('e_month'))
select.select_by_value('12')

select = Select(driver.find_element_by_name('e_day'))
select.select_by_value('31')

driver.find_element_by_css_selector('tbody > tr:nth-child(2) > td > img').click()

In [7]:
driver.switch_to_frame("kako")    # krjmain Frame으로 스위칭 

/home/jongsky/miniconda3/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [8]:
for y in year:
    all_df = pd.DataFrame(columns=['날짜', '평균  기온', '최고  기온', '최저  기온', '강수량(mm)', '신적설(cm)', '평균  풍속(m/s)','평균  습도  (%)', '운량(1/10)', '일조  시간  (Hr)', '날씨'])

    for m in month:
        select = Select(driver.find_element_by_name('start'))
        select.select_by_value('{}-{}-01'.format(y, m))
        driver.find_element_by_css_selector('td:nth-child(2) > img').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('tbody')[2]
        
        df = pd.read_html(str(table),header=0)[0]
        all_df = all_df.append(df)   
        
    all_df = all_df.reset_index(drop=True)
    all_df.to_csv('../data/원본데이터/기상/wetheri_일별기상관측_{}.csv'.format(y))

In [9]:
driver.close()